In [1]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using Test
using ForwardDiff
using SpecialFunctions
using OrthoMatchingPursuit
using BenchmarkTools

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116


In [51]:
Nx = 10
Ne = 500
X = randn(Nx, Ne).^2
Xvalid = randn(Nx, Ne).^2
λ = 0.0
δ = 1e-8
γ = 2.0

2.0

In [60]:
Cgreedy = SparseRadialMapComponent(Nx, -1)
@time Cgreedy, ϵ = optimize(Cgreedy, X, 2, 0, "split", λ, δ, γ; maxpatience = 10^5)

  0.013584 seconds (86.44 k allocations: 10.514 MiB)


(Sparse radial map component of dimension 10 and order p = [2, 2, 2, 2, 2, 2, 2, 2, 2, 0]
, ([0.7656217954159154, 0.7648399506741894, 0.7639387684565466, 0.750167171594771, 0.7377990655730661, 0.7271552118995767, 0.7224895082757199, 0.7216775581211943, 0.7211063766470724, 0.7139435931825915], [0.6702628798909104, 0.6809446280089134, 0.6848954094717498, 0.6844101109929434, 0.6975847745395218, 0.6755273290309295, 0.675904499342735, 0.6808974690887237, 0.6844588382815294, 0.6840312755735277]))

In [62]:
Cgreedy.a

10-element Array{Array{Float64,1},1}:
 [-0.07945095929721153, 2.7114201078599383, -5.187075037774528]
 [-0.12281712321399424, -4.432778594167132, 2.594735292994272]
 [0.008245740918751536, 0.694881411132988, -0.7783505851201096]
 [0.06375966900102961, 0.5129232794603104, 0.1669319761643198]
 [-0.24129945509832884, -3.8107321058081425, 0.8187193204810852]
 [0.004262546684465334, 1.918371406876188, -1.506325621924174]
 [-0.059307979841704266, -1.2797178270300311, 0.7664413724907158]
 [-0.17496480808425596, -3.141802536759589, 0.5862522044731184]
 [-0.12204958599117775, -2.085049694750271, -1.5605416875879632]
 [2.9372211907147214, 0.8073939262600239]

In [63]:
clearcoeff!(Cgreedy)

LoadError: [91mMethodError: no method matching zero(::Type{Array{Float64,1}})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  zero([91m::Type{Pkg.Resolve.VersionWeight}[39m) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/Pkg/src/Resolve/versionweights.jl:15[39m
[91m[0m  zero([91m::Type{Pkg.Resolve.FieldValue}[39m) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/Pkg/src/Resolve/fieldvalues.jl:38[39m
[91m[0m  zero([91m::Type{Dates.Date}[39m) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/Dates/src/types.jl:405[39m
[91m[0m  ...[39m

In [19]:
@time Cgreedy, error = greedyfit(Nx, 2, 2, deepcopy(X), 3, λ, δ, γ)

  0.018648 seconds (91.66 k allocations: 11.540 MiB)


(Sparse radial map component of dimension 5 and order p = [2, -1, 2, 2, 2]
, [0.16149625717495586, 0.15955843305309023, 0.1513458062113992, 0.14608833904374321])

In [20]:
@time Cgreedy, error = greedyfit(Nx, 2, 0, deepcopy(X), deepcopy(Xvalid), 4, λ, δ, γ; verbose = false)

  0.004808 seconds (54.84 k allocations: 5.174 MiB)


(Sparse radial map component of dimension 5 and order p = [2, 2, 2, 2, 0]
, ([0.9522111470540667, 0.9468735067392048, 0.9382460125763467, 0.9339206542025913, 0.9330111942049354], [0.8123465391957869, 0.8138996919068819, 0.8178551591012666, 0.8222529910064743, 0.8217679113283066]))

In [33]:
C = SparseRadialMapComponent(Nx, Cgreedy.p)

Sparse radial map component of dimension 5 and order p = [1, 1, 1, 1, 0]


In [34]:
@time center_std!(C, sort(deepcopy(X); dims = 2); γ = γ)

  0.007385 seconds (249 allocations: 114.234 KiB)


In [35]:
@time x_opt = optimize(C, X, λ, δ)
modify_a!(C, x_opt)

  0.000287 seconds (2.58 k allocations: 277.781 KiB)


In [36]:
negative_likelihood(C, X)

0.9143832086452744

In [37]:
negative_likelihood(Cgreedy, X)

0.9143832086452746